# TME 2 

## 1. Introduction

In [1]:
import pandas as pd
import itertools as it
import numpy as np

In [2]:
M1 = [[0,8,7,12], [8,0,9,14], [7,9,0,11], [12,14,11,0]]
M2 = [[0,2,3,8,14,18],[2,0,3,8,14,18],
      [3,3,0,8,14,18],[8,8,8,0,14,18],
      [14,14,14,14,0,18],[18,18,18,18,18,0]]
#UPGMA
M3 = [[0,19,27,8,33,18,13],[19,0,31,18,36,1,13],
          [27,31,0,26,41,32,29],[8,18,26,0,31,17,14],
          [33,36,41,31,0,35,28],[18,1,32,17,35,0,12],
          [13,13,29,14,28,12,0]]
#Neighbor Joining
M4 = [[0,2,4,6,6,8],[2,0,4,6,6,8],
          [4,4,0,6,6,8],[6,6,6,0,4,8],
          [6,6,6,4,0,8],[8,8,8,8,8,0]]

M1D = pd.DataFrame(M1)
M2D = pd.DataFrame(M2)
UPGMAD = pd.DataFrame(M3) 
NJD = pd.DataFrame(M4)

M1D
UPGMAD

,0,1,2,3,4,5,6
0,0,19,27,8,33,18,13
1,19,0,31,18,36,1,13
2,27,31,0,26,41,32,29
3,8,18,26,0,31,17,14
4,33,36,41,31,0,35,28
5,18,1,32,17,35,0,12
6,13,13,29,14,28,12,0


###### Question 1

##### Matrice additive

Une est matrice additive si et seulement si la condition des 4 points est vraie pour tous les quartets i,j,k et l où 1<=i,j,k,l<=n).
La condition des 4 points est remplie quand Dij + Dkl <= max(Dik+Djl,Dil+Djk).

In [3]:
def estadditive(M):
    comb=it.combinations(range(len(M)),4)
    for groupe in comb:
        print(groupe)
        i,j,k,l=groupe
        if not (M[i,j]+M[k,l]<=max(M[i,k]+M[j,l],M[i,l]+M[j,k])):
            return False
    return True

In [4]:
M1 = [[0,8,7,12], [8,0,9,14], [7,9,0,11], [12,14,11,0]]
M1 = np.array(M1)
print(estadditive(M1))

(0, 1, 2, 3)
True


##### Ultramétricité

Une matrice est dite ultramétrique quand, pour tout choix de 3 points i,j,k au moins deux distances sont égales : Dij et Djk.

On a alors mathématiquement : Dij = Djk >Dik

Une matrice ultramétrique est forcément additive.

In [5]:
def estultra(M):
    comb=it.combinations(range(len(M)),3)
    for groupe in comb:
        i,j,k=groupe
        if not(M[i,k]<=max(M[i,j],M[j,k])):
            return False
    return True

In [6]:
M1 = np.array(M1)
print(estultra(M1))

False


###### Question 2

In [7]:
def cluster(M,i):            #i = l'espèce en question
    return np.sum(M[i])

In [8]:
def groupe(M):
    L=[]
    for k in range(len(M)):
        L.append(cluster(M,k))
    return L

In [9]:
print(cluster(M1,1))
print(groupe(M1))

31
[27, 31, 27, 37]


## 2. UPGMA

###### Question 1 

Le format Newick est un format de fichier utilisé en phylogénie pour décrire les relations phylogénétiques entre différents clusters.
Par exemple si on a un ancêtre commun entre A et B puis entre AB et C on écrira sous ce format :

(C : longueur de la branche (A : longueur, B : longueur) : longueur restante de la branche)

L'extension de ce format est .nwk

###### Question 2

In [45]:
def UPGMA(M):
    Alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWYZ'
    nodes = 'abcdefghijklm'
    nodes = [i for i in nodes]
    M1 = np.copy(M)
    n=len(M1)
    Cardinal = [1 for k in range(n)]
    Clusters = [Alphabet[k] for k in range(n)]
    Newick = ''
    arbre = {}
    for i in range(len(M1)):
        for j in range(len(M1[i])):
            if i==j:
                M1[i,j] = 1000
    while len(M1)>1:
        Indice = np.argmin(M1)
        i,j = np.unravel_index(np.argmin(M1), M1.shape)  #trouve la ligne et la colonne correspondant au argmin
        d = np.round(M1[i,j]/2,1)

        for l in range(len(M1)):
            if l!=i and l!=j:
                M1[i,l] = (M1[i,l]*Cardinal[i]+M1[j,l]*Cardinal[l])/(Cardinal[i]+Cardinal[l])
                M1[l,i] = M1[i,l]
        
        
        
        parent = nodes.pop(0)
        child1, child2 = Clusters[i], Clusters[j]
        di = d         
            
        while child1 in arbre or child2 in arbre:         # tant que un des enfants est un noeud on n'a pas atteint la longueur 
            if child1 in arbre:
                print('add1', arbre[child1][1][1])
                print('d avant:',di)
                di += arbre[child1][1][1]
                print('d après', di)
                child1,child2 = arbre[child1][1][0], arbre[child1][2][0]
            elif child2 in arbre:
                print('add2', arbre[child2][1][1])
                print('d avant:',di)
                di += arbre[child2][1][1]
                print('d après', di)
                child1,child2 = arbre[child2][1][0], arbre[child2][2][0]
                
        if Clusters[i] in arbre and not Clusters[j] in arbre :
            arbre[parent] = (Cardinal[i]+Cardinal[j], (Clusters[i], d), (Clusters[j], di))
        elif Clusters[j] in arbre and not Clusters[i] in arbre:
            arbre[parent] = (Cardinal[i]+Cardinal[j], (Clusters[i], di), (Clusters[j], d))
        else:
            arbre[parent] = (Cardinal[i]+Cardinal[j], (Clusters[i], d), (Clusters[j], d))
        print(arbre)
                
        Clusters[i] = parent
        del Clusters[j]
        Cardinal[i] += Cardinal[j]
        del Cardinal[j]
        M1 = np.delete(M1,j,0)
        M1 = np.delete(M1,j,1)

    return M1, arbre, Clusters[0]

In [46]:
M3 = np.array(M3,dtype=float)
UPGMAD
print(UPGMA(M3)[0])

{'a': (2, ('B', 0.5), ('F', 0.5))}
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0))}
add1 0.5
d avant: 6.2
d après 6.7
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0)), 'c': (3, ('a', 6.2), ('G', 6.7))}
add1 4.0
d avant: 7.8
d après 11.8
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0)), 'c': (3, ('a', 6.2), ('G', 6.7)), 'd': (5, ('b', 7.8), ('c', 7.8))}
add1 7.8
d avant: 13.9
d après 21.7
add1 4.0
d avant: 21.7
d après 25.7
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0)), 'c': (3, ('a', 6.2), ('G', 6.7)), 'd': (5, ('b', 7.8), ('c', 7.8)), 'e': (6, ('d', 13.9), ('C', 25.7))}
add1 13.9
d avant: 16.9
d après 30.799999999999997
add1 7.8
d avant: 30.799999999999997
d après 38.599999999999994
add1 4.0
d avant: 38.599999999999994
d après 42.599999999999994
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0)), 'c': (3, ('a', 6.2), ('G', 6.7)), 'd': (5, ('b', 7.8), ('c', 7.8)), 'e': (6, ('d', 13.9), ('C'

In [47]:
def pile_str_modifie(arbre, parent, leaf):
    """
    Fonction qui convertit l'arbre de l'algorithme NJ
    """
    Res = ''
    pile = [ parent ]
    while pile != []:
        ref = pile.pop(-1)
        if ref in arbre.keys():
            pile.append(')')
            pile.append(':' + str(arbre[ref][2][1]))
            pile.append(arbre[ref][2][0])
            pile.append(',')
            pile.append(':' + str(arbre[ref][1][1]))
            pile.append(arbre[ref][1][0])
            pile.append('(')
        else:
            Res += ref
    return Res

feuilles = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
mat, arbre, racine = UPGMA(M3)
res = pile_str_modifie(arbre, racine, feuilles)
print('res', res)

{'a': (2, ('B', 0.5), ('F', 0.5))}
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0))}
add1 0.5
d avant: 6.2
d après 6.7
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0)), 'c': (3, ('a', 6.2), ('G', 6.7))}
add1 4.0
d avant: 7.8
d après 11.8
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0)), 'c': (3, ('a', 6.2), ('G', 6.7)), 'd': (5, ('b', 7.8), ('c', 7.8))}
add1 7.8
d avant: 13.9
d après 21.7
add1 4.0
d avant: 21.7
d après 25.7
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0)), 'c': (3, ('a', 6.2), ('G', 6.7)), 'd': (5, ('b', 7.8), ('c', 7.8)), 'e': (6, ('d', 13.9), ('C', 25.7))}
add1 13.9
d avant: 16.9
d après 30.799999999999997
add1 7.8
d avant: 30.799999999999997
d après 38.599999999999994
add1 4.0
d avant: 38.599999999999994
d après 42.599999999999994
{'a': (2, ('B', 0.5), ('F', 0.5)), 'b': (2, ('A', 4.0), ('D', 4.0)), 'c': (3, ('a', 6.2), ('G', 6.7)), 'd': (5, ('b', 7.8), ('c', 7.8)), 'e': (6, ('d', 13.9), ('C'

In [ ]:
'(((((F:0.5,B:0.5):5.75,G:6.25):3.25,(D:4.0,A:4.0):5.5):3.5,C:13.0):4.0,E:17.0)'

In [ ]:
UPGMAD = pd.DataFrame(UPGMA(M3))
UPGMAD

## 3. NJ

##### Question 1

In [ ]:
def U(M,i,N):      #i est l'indice sur les lignes représente un cluster donné
    u=0
    n=len(M)
    for j in range (n):
        u+=M[i,j]/(N-2)
    return u

In [ ]:
print(U(M3,0,7))
print(M3)

##### Question 2

In [ ]:
def Q(M,i,j,n):
    Ui=U(M,i,n)
    Uj=U(M,j,n)
    return M[i,j]-Ui-Uj

In [ ]:
print(Q(M3,1,5,7))

##### Question 3

In [ ]:
def NJ(M):
    Alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    nodes = 'abcdefghijklmopqrstuvwxyz'
    nodes = [i for i in nodes]
    N=len(M)
    Clusters = [Alphabet[i] for i in range(N)]
    print('Clusters', Clusters)
    M1 = np.copy(M)
    arbre = {}
    for k in range(len(M1)):
        M1[k,k]=1000
    while len(M1)>1:
        n = len(M1)
        MQ = np.zeros((n,n),dtype='float')
        for k in range(n):
            MQ[k,k] = 1000
        comb = it.combinations(range(n),2)
        for groupe in comb:
            i, j = groupe
            MQ[i,j] = Q(M1,i,j,N)
            MQ[j,i] = Q(M1,i,j,N)
        i,j = np.unravel_index(np.argmin(MQ), MQ.shape)
        Ui = U(M1,i,N)
        Uj = U(M1,j,N)
        d = M1[i,j]
        Di = (d+(Ui-Uj))/2
        Dj = (d+(Uj-Ui))/2
        
        print('i', i)
        print('j', j)
        parent = nodes.pop(0)
        arbre[parent] = (0, (Clusters[i],Di), (Clusters[j],Dj))
        
        Clusters[i] = parent
        print(j, Clusters[j])
        Clusters.pop(j)
        print('Clusters', Clusters)
        
        for l in range(len(M1)):
            if l!=i and l!=j:
                M1[i,l] = (M1[i,l]+M1[j,l]-d)/2
                M1[l,i] = M1[i,l]
        M1 = np.delete(M1,j,0)
        M1 = np.delete(M1,j,1)
        print(M1)
    return arbre, Clusters[0]
    

In [ ]:
print(NJ(M3))

In [ ]:
def pile_str_modifie(arbre, parent, leaf):
    """
    Fonction qui convertit l'arbre de l'algorithme NJ
    """
    Res = ''
    pile = [ parent ]
    while pile != []:
        ref = pile.pop(-1)
        if ref in arbre.keys():
            pile.append(')')
            pile.append(':' + str(arbre[ref][2][1]))
            pile.append(arbre[ref][2][0])
            pile.append(',')
            pile.append(':' + str(arbre[ref][1][1]))
            pile.append(arbre[ref][1][0])
            pile.append('(')
        else:
            Res += ref
    return Res

tree, parent = NJ(M3)

In [ ]:
Alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
feuilles = [Alphabet[i] for i in range(len(M3))]
print(tree)
Newick = pile_str_modifie(tree, parent , feuilles)
print(Newick)